In [1]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import time
%matplotlib inline

In [2]:

def imshow(inp):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
   

## Peep look into the downloaded data 

```
    chapter3/
        dogsandcats/
            train/
                dog.183.jpg
                cat.2.jpg
                cat.17.jpg
                dog.186.jpg
                cat.27.jpg
                dog.193.jpg
  

```

```
    chapter3/
        dogsandcats/
            train/
                dog/
                    dog.183.jpg
                    dog.186.jpg
                    dog.193.jpg
                cat/
                    cat.17.jpg
                    cat.2.jpg
                    cat.27.jpg
            valid/
                dog/
                    dog.173.jpg
                    dog.156.jpg
                    dog.123.jpg
                cat/
                    cat.172.jpg
                    cat.20.jpg
                    cat.21.jpg

```



## Create validation data set

In [3]:
# path = '../chapter3/dogsandcats/'
path = '../data/dogsandcats/'

In [6]:
# files = glob(os.path.join(path,'*/*.jpg'))
files = glob(os.path.join(path,'train','*.jpg'))
# Note : 'iglob' which returns an iterator, instead of loading the names into memory

In [7]:
# file list test
idx_start = 0
files[idx_start:idx_start+10]

['../data/dogsandcats/train/dog.3270.jpg',
 '../data/dogsandcats/train/dog.4531.jpg',
 '../data/dogsandcats/train/dog.8351.jpg',
 '../data/dogsandcats/train/dog.10148.jpg',
 '../data/dogsandcats/train/cat.2969.jpg',
 '../data/dogsandcats/train/dog.6459.jpg',
 '../data/dogsandcats/train/dog.9689.jpg',
 '../data/dogsandcats/train/cat.4104.jpg',
 '../data/dogsandcats/train/dog.10577.jpg',
 '../data/dogsandcats/train/cat.9944.jpg']

In [8]:
# glob test
dirs = glob(os.path.join(path,'*/'))
dirs[:]

['../data/dogsandcats/train/']

In [9]:
print(f'Total no of images {len(files)}') # train + test

Total no of images 25000


In [10]:
no_of_images = 25000 # train?

In [11]:
no_of_images = len(files)

In [12]:
no_of_images*0.8

20000.0

In [13]:
shuffle = np.random.permutation(no_of_images)

In [14]:
# np.random.permutation test
len(shuffle)

25000

In [15]:
# np.random.permutation test
shuffle[:10]

array([19009, 13312, 11701,  1547, 22794,  3439,  7847,  2827,  6307,
        5072])

In [16]:
os.mkdir(os.path.join(path,'valid'))
# if folder exist gives error

In [17]:
for t in ['train','valid']:
    for folder in ['dog/','cat/']:
        os.mkdir(os.path.join(path,t,folder)) 


In [18]:
# test for 'split'
# sample_folder = files[shuffle[10]].split('/')[-1].split('.')[0]
print(files[shuffle[1]])
print(files[shuffle[1]].split('/'))
print(files[shuffle[1]].split('/')[-1])
print(files[shuffle[1]].split('/')[-1].split('.'))
print(files[shuffle[1]].split('/')[-1].split('.')[0])

../data/dogsandcats/train/dog.3093.jpg
['..', 'data', 'dogsandcats', 'train', 'dog.3093.jpg']
dog.3093.jpg
['dog', '3093', 'jpg']
dog


In [19]:
for i in shuffle[:2000]:
    #shutil.copyfile(files[i],'../chapter3/dogsandcats/valid/')
    folder = files[i].split('/')[-1].split('.')[0] # extract label from filename 'dog' or 'cat'
    image = files[i].split('/')[-1]
    os.rename(files[i],os.path.join(path,'valid',folder,image))


In [20]:
for i in shuffle[2000:]:
    #shutil.copyfile(files[i],'../chapter3/dogsandcats/valid/')
    folder = files[i].split('/')[-1].split('.')[0]
    image = files[i].split('/')[-1]
    os.rename(files[i],os.path.join(path,'train',folder,image))



In [21]:
print(len(glob(os.path.join(path,'valid','dog','*.jpg'))))
print(len(glob(os.path.join(path,'valid','cat','*.jpg'))))

1020
980


In [22]:
print(len(glob(os.path.join(path,'train','dog','*.jpg'))))
print(len(glob(os.path.join(path,'train','cat','*.jpg'))))

11480
11520


## Check if GPU is present

In [23]:
if torch.cuda.is_available():
    print("cuda is available")
    is_cuda = True

cuda is available


# If you want to start from training netwok , run below codes from here
---

In [1]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import time
import copy
%matplotlib inline

In [2]:
path = '../data/dogsandcats/'

## Load data into PyTorch tensors

In [3]:
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train = ImageFolder('../data/dogsandcats/train/',simple_transform)
valid = ImageFolder('../data/dogsandcats/valid/',simple_transform)

In [4]:
print(train.class_to_idx)
print(train.classes) 


{'cat': 0, 'dog': 1}
['cat', 'dog']


In [5]:
# def imshow(inp):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)

imshow(train[22999][0])

NameError: name 'imshow' is not defined

# Training network

## Create data generators

In [28]:
batch_size = 64
nCPUcores = 4 
# train_data_gen = torch.utils.data.DataLoader(train,shuffle=True,batch_size=batch_size,num_workers=nCPUcores)
# valid_data_gen = torch.utils.data.DataLoader(valid,batch_size=batch_size,num_workers=nCPUcores)

train_data_gen = torch.utils.data.DataLoader(train,batch_size=64,num_workers=3)
valid_data_gen = torch.utils.data.DataLoader(valid,batch_size=64,num_workers=3)

In [29]:
dataset_sizes = {'train':len(train_data_gen.dataset),'valid':len(valid_data_gen.dataset)}

In [30]:
print(dataset_sizes['train'])
print(dataset_sizes['valid'])

23000
2000


In [31]:
dataloaders = {'train':train_data_gen,'valid':valid_data_gen}

## Create a network

In [32]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

In [33]:
if torch.cuda.is_available():
    print("Loading network model to GPU")
    model_ft = model_ft.cuda()

Loading network model to GPU


In [34]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [35]:
# Loss and Optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [51]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if torch.cuda.is_available():
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                    
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
#                 loss = criterion(outputs, labels)
                loss = criterion(preds, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
#                 running_loss += loss.data[0]
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [52]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=2)

Epoch 0/1
----------


RuntimeError: log_softmax_forward is not implemented for type torch.cuda.LongTensor